In [1]:
from unityagents import UnityEnvironment
import numpy as np
from dqn_agent import Agent
from collections import deque
import torch
import matplotlib.pyplot as plt

In [2]:
env = UnityEnvironment(file_name="C:/Users/DNEUHAE/Desktop/DRL/p1_navigation/Banana_Windows_x86_64/Banana.exe")
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
action_size = brain.vector_action_space_size
state_size = brain.vector_observation_space_size
agent = Agent(state_size=action_size, action_size=state_size, seed=0)

In [4]:
#### TRAINING AGENT ####
num_episodes = 2000
epsilon = 1.0
epsilon_decay = 0.995 #1/1000
epsilon_min = 0.01 #0.1

scores = []
last_mean_score = 13

for episode in range(num_episodes):
    
    env_info = env.reset(train_mode=True)[brain_name]           # reset the environment
    state = env_info.vector_observations[0]                     # get the current state
    score = 0                                                   # initialize the score

    while True:
        action = agent.act(state, epsilon)
        env_info = env.step(action)[brain_name]
        next_state = env_info.vector_observations[0]            # get the next state
        reward = env_info.rewards[0]                            # get the reward
        done = env_info.local_done[0]                           # see if episode has finished
        agent.step(state, action, reward, next_state, done)     # update local Q-net, and soft update target Q-net
        score += reward                                         # update the score                                          
        state = next_state                                      # roll over the state to next time step
        if done:                                                # exit loop if episode finished
            scores.append(score)
            break

    epsilon = max(epsilon_decay**episode, epsilon_min)  #max(epsilon-epsilon_decay, epsilon_min)   # adjust epsilon
    print('Episode:', episode, 'Epsilon:', epsilon, '- Score:', score)
    
    if len(scores) >= 100:
        mean_score = np.mean(scores[-100:])
        if mean_score > last_mean_score:
            torch.save(agent.qnetwork_local.state_dict(), 'exponentialEpsilon_doubleDQN_64x64_'+str(episode)+'.pth')
            last_mean_score = mean_score

    
env.close()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x37 and 4x64)